In [ ]:
%pip install --force-reinstall   torch torchvision  torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

In [ ]:
%cd /notebooks/
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!git pull
%pip install -e .
%cd /notebooks/

In [ ]:
%pip install git+https://github.com/speechbrain/speechbrain/

In [ ]:
%pip install  librosa  --upgrade 

In [ ]:
%pip install requests moviepy librosa pytube pydub

In [ ]:
%pip install git+https://github.com/espnet/espnet
%pip install -q espnet_model_zoo

In [ ]:
from pydub import AudioSegment
import os

def download_video_at_url(url,output_path):
    # Utilisez la commande ! pour télécharger le fichier en utilisant aria2c
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o {output_path} {url}
def split_audio_into_chunks(file_path, output_dir,minutes=5):
    """
    Splits an audio file into five-minute chunks and saves them to the output directory.
    
    :param file_path: The path to the input audio file.
    :param output_dir: The directory where the audio chunks will be saved.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Load the audio file
    audio = AudioSegment.from_file(file_path)
    
    # Define the length of each chunk in milliseconds (5 minutes)
    chunk_length_ms = minutes * 60 * 1000
    
    # Split the audio into chunks
    chunks = []
    for i in range(0, len(audio), chunk_length_ms):
        chunk = audio[i:i + chunk_length_ms]
        chunks.append(chunk)
    
    # Export each chunk as a separate audio file
    for i, chunk in enumerate(chunks):
        chunk_filename = f"{output_dir}/chunk_{i}.wav"  # You can change the format to '.mp3' if needed
        chunk.export(chunk_filename, format="wav")  # Change 'wav' to 'mp3' if exporting as MP3
        print(f"Exported {chunk_filename}")



# Function to download YouTube video
def download_youtube_video(youtube_url, output_path):
    !/root/miniconda3/envs/tts/bin/yt-dlp  -o {output_path} {youtube_url}
    return output_path

# Function to extract audio from video
def extract_audio_from_video(video_path, audio_output_path):
    with VideoFileClip(video_path) as video:
        audio = video.audio
        audio.write_audiofile(audio_output_path, codec='aac')

# Function to resample audio
def resample_audio(input_audio_path, output_audio_path, target_sr):
    signal, sr_orig = librosa.load(input_audio_path, sr=None)  # Load audio without resampling
    signal_resampled = librosa.resample(signal, orig_sr=sr_orig, target_sr=target_sr)  # Resample audio
    sf.write(output_audio_path, signal_resampled, target_sr)  # Save resampled audio


In [ ]:
from pytube import YouTube
from moviepy.editor import VideoFileClip
import librosa
import soundfile as sf
from IPython.display import Audio

# URL of the YouTube video
youtube_url = 'https://www.youtube.com/watch?v='

# Paths for the downloaded video and audio files
video_path = 'downloaded_video.mp4'
audio_path = 'extracted_audio.aac'
resampled_audio_path = 'resampled_audio.wav'




In [ ]:

!rm /notebooks/{video_path}
# Download the video from YouTube
download_youtube_video(youtube_url, video_path)

In [ ]:
# Target sampling rate
target_sampling_rate = 24000

# Extract audio from the downloaded video
extract_audio_from_video(video_path, audio_path)

# Resample the extracted audio to the target sampling rate
resample_audio(audio_path, resampled_audio_path, target_sampling_rate)

print(f"Resampled audio saved to: {resampled_audio_path}")

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
%pip install pyannote.audio

In [ ]:
!rm -rf ./reseampled_audio
split_audio_into_chunks('resampled_audio.wav', './reseampled_audio',2)

In [ ]:
from huggingface_hub import login
login(token='',add_to_git_credential=True)

In [ ]:
import librosa
import soundfile as sf

# Load your audio file (replace 'your_audio_file.wav' with your file path)
y, sr = librosa.load('resampled_audio.wav', sr=None)  # 'sr=None' loads the file in its original sampling rate

# Resample the audio to 48000 Hz
y_48000 = librosa.resample(y,orig_sr=sr,target_sr=48000)

# Save the resampled audio (replace 'resampled_audio.wav' with your desired output file path)
sf.write('resampled_audio48.wav', y_48000, 48000)

In [ ]:
from df.enhance import enhance, init_df, load_audio, save_audio
from df.utils import download_file

# Load default model
model, df_state, _ = init_df()
# Download and open some audio file. You use your audio files here
audio_path = 'resampled_audio48.wav'
audio, _ = load_audio(audio_path, sr=df_state.sr())
# Denoise the audio
enhanced = enhance(model, df_state, audio)
# Save for listening
save_audio("enhanced.wav", enhanced, df_state.sr())

In [ ]:
from pyannote.audio import Pipeline, Audio
from pydub import AudioSegment
import torch
import os

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1")
pipeline.to(torch.device("cuda"))
audio_path="enhanced.wav"
# apply pretrained pipeline
diarization = pipeline(audio_path)
audio = AudioSegment.from_wav(audio_path)
output_dir = "by_speaker"
!rm -rf {output_dir}
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

speakers = {}
for turn, _, speaker in diarization.itertracks(yield_label=True):
    start_ms = int(turn.start * 1000)
    end_ms = int(turn.end * 1000)
    
    speaker_audio = audio[start_ms:end_ms]
    if speaker not in speakers:
        speakers[speaker] = speaker_audio
    else:
        speakers[speaker] += speaker_audio

for speaker, audio in speakers.items():
    audio.export(os.path.join(output_dir, f"{speaker}.wav"), format="wav")

In [ ]:
!rm -rf ./reseampled_audio
split_audio_into_chunks('/notebooks/by_speaker/SPEAKER_01.wav', './reseampled_audio',2)

In [ ]:
from pydub import AudioSegment

# Load audio file
audio = AudioSegment.from_wav("/notebooks/by_speaker/SPEAKER_01.wav") 

# Get start and end times in milliseconds
start_time = 0.05*60*1000 + 0*1000 # 2 min 54 sec
end_time = 0.15*60*1000 + 0*1000  # 4 min

# Extract segment
segment = audio[start_time:end_time]

# Resample to 24000 Hz
segment = segment.set_frame_rate(16000)

# Convert to mono
segment = segment.set_channels(1)

# Export to wav file
segment.export("segment_24kmono.wav", format="wav")

In [ ]:
from espnet_model_zoo.downloader import ModelDownloader

d = ModelDownloader()
cfg = d.download_and_unpack("espnet/Wangyou_Zhang_chime4_enh_train_enh_conv_tasnet_raw")

In [ ]:
import os

import soundfile
from IPython.display import display, Audio
mixwav_mc, sr = soundfile.read("/notebooks/by_speaker/SPEAKER_00.wav")
# mixwav.shape: num_samples, num_channels
mixwav_sc = mixwav_mc[:]
display(Audio(mixwav_mc.T, rate=sr))

In [ ]:
import sys
import soundfile
from espnet2.bin.enh_inference import SeparateSpeech


separate_speech = {}
# For models downloaded from GoogleDrive, you can use the following script:
enh_model_sc = SeparateSpeech(
  train_config=cfg["train_config"],
  model_file=cfg["model_file"],
  # for segment-wise process on long speech
  normalize_segment_scale=True,
  show_progressbar=True,
  ref_channel=1,
  normalize_output_wav=True,
  device="cuda:0",
)

In [ ]:

# play the enhanced single-channel speech
wave = enh_model_sc(mixwav_sc[None, ...], sr)
print("Input real noisy speech", flush=True)
display(Audio(mixwav_sc, rate=sr))
print("Enhanced speech", flush=True)
display(Audio(wave[0].squeeze(), rate=sr))

In [ ]:
soundfile.write("enhanced.wav", wave[0].squeeze(), sr)

In [ ]:
from IPython.display import Audio

Audio('enhanced.wav')


In [ ]:
from IPython.display import Audio
Audio('/notebooks/by_speaker/SPEAKER_00.wav')

In [ ]:
from IPython.display import Audio
Audio('enhanced.wav')